<a href="https://colab.research.google.com/github/johnsDevassy/Worked_Projects/blob/master/Medical_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#lets take the data
import nltk
import os
import zipfile
ziploc="/content/drive/My Drive/datasets/medicalnotes-2019.zip"
zipref=zipfile.ZipFile(ziploc,'r')
zipref.extractall("/content/datas")
from nltk.corpus import stopwords
nltk.download('punkt') 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
stemmer = WordNetLemmatizer()
nltk.download('wordnet') 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
#lets make the Training data
train_zip_ref=zipfile.ZipFile("/content/datas/data_split.zip",'r')
train_zip_ref.extractall("/content/datas/collection")

In [ ]:
dirnames=os.listdir("/content/datas/collection/dataset/train-data")
dirnames

['Urology', 'Gastroenterology', 'Orthopedic', 'Radiology', 'Neurology']

In [ ]:
#lets add labels for the data
import pandas as pd
import io
data_label=pd.DataFrame(columns={'target','zcontent'})
import codecs
con=0
for i in dirnames:
  filenames=os.listdir("/content/datas/collection/dataset/train-data/"+i)
  for j in filenames:
    f=io.open("/content/datas/collection/dataset/train-data/"+i+"/"+j,'r',encoding='cp1252')
    content=f.read()
    
    data_label=data_label.append({'target':con,'zcontent':content},ignore_index=True)
  con+=1

In [ ]:
#Haha We created our train labels
data_label

,zcontent,target
0,\n\n\n\n\n<B>PREOPERATIVE DIAGNOSIS: </B> Righ...,0
1,\n\n\n\n\n<B>CHIEF COMPLAINT:</B> The patient...,0
2,\n\n\n\n\n<B>EXAM: </B> Ultrasound examination...,0
3,\n\n\n\n\n<B>HISTORY OF PRESENT ILLNESS:</B> ...,0
4,\n\n\n\n\n<B>PREOPERATIVE DIAGNOSES:</B>\n1. ...,0
...,...,...
821,\n\n\n\n\n<B>CHIEF COMPLAINT: </B> Followup of...,4
822,\n\n\n\n\n<B>EXAM:</B> CT cervical spine (C-s...,4
823,\n\n\n\n\n<B>REASON FOR VISIT:</B> Followup s...,4
824,\n\n\n\n\n<B>DIAGNOSIS:</B> Status post brain...,4


In [ ]:
def preprocesso(need):  
  #lets import and preprocess our train data
  import nltk
  #nltk.download('stopwords')
  stop_words = set(stopwords.words('english'))  
  from nltk.tokenize import word_tokenize
  import string
  import re
  s=re.sub("<B>.*</B>","",need)
  s=re.sub("\n"," ",s)
  mains="ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz "
  news=""
  for i in s:
    if i in mains:
      news=news+i
  news=re.sub(" +"," ",news)
  newstr=""
  for i in news.split():
    if i not in stop_words and len(i)>2:
      newstr=newstr+" "+i
  tok=[stemmer.lemmatize(word) for word in newstr.split()]
  return tok


In [ ]:
data_label.zcontent=data_label.zcontent.apply(preprocesso)

In [ ]:
!pip install --upgrade gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.8.3)


In [ ]:
from gensim.models import Word2Vec , KeyedVectors

In [ ]:
model=Word2Vec(data_label.zcontent.to_list(),min_count=1,size=32)

In [ ]:
for i in data_label.index:
  tex=data_label.zcontent[i]
  vec=[]
  
  for j in tex:
    vec.append(model[j].tolist())
  data_label.zcontent[i]=vec

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [ ]:
data_label

,zcontent,target
0,"[[-0.5668182969093323, 1.179060935974121, -0.0...",0
1,"[[-1.4447742700576782, 0.37954527139663696, -0...",0
2,"[[-0.15619951486587524, 0.15895548462867737, -...",0
3,"[[-1.4447742700576782, 0.37954527139663696, -0...",0
4,"[[-0.13951276242733002, 0.1748700588941574, -0...",0
...,...,...
821,"[[-0.21859118342399597, 0.1736731380224228, -0...",4
822,"[[-0.57984459400177, 1.5192397832870483, 0.135...",4
823,"[[-0.21859118342399597, 0.1736731380224228, -0...",4
824,"[[-0.4724074602127075, 0.4584673345088959, -0....",4


In [ ]:
#PLAYte5Ud9ieQqe

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_label.zcontent, data_label.target, test_size=0.1)

In [ ]:
X_train , y_train

(757    [[-0.08747728914022446, 0.078652523458004, -0....
 117    [[-0.10850756615400314, 0.08147795498371124, -...
 21                                                    []
 108    [[-0.08296667784452438, 0.04786066710948944, -...
 484    [[-1.4447742700576782, 0.37954527139663696, -0...
                              ...                        
 461    [[-0.04153839126229286, 0.04401073232293129, 0...
 600    [[-0.8572915196418762, 0.963405966758728, -0.4...
 429    [[-0.1359962671995163, 0.14434345066547394, -0...
 562    [[-0.2506360411643982, 0.3022376596927643, -0....
 644    [[-0.11611883342266083, 0.09996725618839264, -...
 Name: zcontent, Length: 743, dtype: object, 757    4
 117    1
 21     0
 108    0
 484    2
       ..
 461    2
 600    3
 429    2
 562    3
 644    3
 Name: target, Length: 743, dtype: object)

In [ ]:
from sklearn.preprocessing import LabelEncoder
Encoder = LabelEncoder()
y_train = Encoder.fit_transform(y_train)
y_test = Encoder.fit_transform(y_test)

In [ ]:
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB() 
gnb.fit(X_train, y_train) 

ValueError: ignored

In [ ]:
from sklearn.naive_bayes import MultinomialNB

NB_classifier = MultinomialNB()
NB_classifier.fit(X_train, y_train)

ValueError: ignored

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
y_predict_test = NB_classifier.predict(X_test)
cm = confusion_matrix(y_test, y_predict_test)
sns.heatmap(cm, annot=True)

print(classification_report(y_test, y_predict_test))
